In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [8]:
#load data
#this might take a while as it will download the dataset from internet
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
data_train = torchvision.datasets.MNIST('./', download=True, train=True, transform = transform)
data_test = torchvision.datasets.MNIST('./', download=True, train=False, transform = transform)

100.0%


Extracting ./MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST\raw



102.8%


Extracting ./MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST\raw



0.5%

...

Extracting ./MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST\raw



112.7%

Extracting ./MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST\raw

Processing...



C:\Users\smann\anaconda3\envs\fri_rl\lib\site-packages\torchvision\datasets\mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [22]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(data_train, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(data_test, batch_size=batch_size, shuffle=True)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [23]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [24]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9])


In [25]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [26]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [27]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300897  [    0/60000]
loss: 2.291789  [ 6400/60000]
loss: 2.302828  [12800/60000]
loss: 2.279132  [19200/60000]
loss: 2.283685  [25600/60000]
loss: 2.279066  [32000/60000]
loss: 2.239423  [38400/60000]
loss: 2.245624  [44800/60000]
loss: 2.228656  [51200/60000]
loss: 2.240869  [57600/60000]
Test Error: 
 Accuracy: 27.3%, Avg loss: 0.034693 

Epoch 2
-------------------------------
loss: 2.206742  [    0/60000]
loss: 2.182684  [ 6400/60000]
loss: 2.188540  [12800/60000]
loss: 2.163846  [19200/60000]
loss: 2.171321  [25600/60000]
loss: 2.130929  [32000/60000]
loss: 2.045654  [38400/60000]
loss: 2.074921  [44800/60000]
loss: 1.928407  [51200/60000]
loss: 2.042920  [57600/60000]
Test Error: 
 Accuracy: 34.1%, Avg loss: 0.031730 

Epoch 3
-------------------------------
loss: 2.072551  [    0/60000]
loss: 2.100689  [ 6400/60000]
loss: 1.919591  [12800/60000]
loss: 1.782083  [19200/60000]
loss: 1.940554  [25600/60000]
loss: 1.730199  [32000/600

In [28]:
classes = [
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
]

model.eval()
for t in range(len(data_test)):
    x, y = data_test[t][0], data_test[t][1]
    with torch.no_grad():
        pred = model(x)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "7", Actual: "7"
Predicted: "2", Actual: "2"
Predicted: "2", Actual: "1"
Predicted: "0", Actual: "0"
Predicted: "4", Actual: "4"
Predicted: "2", Actual: "1"
Predicted: "4", Actual: "4"
Predicted: "9", Actual: "9"
Predicted: "2", Actual: "5"
Predicted: "9", Actual: "9"
Predicted: "0", Actual: "0"
Predicted: "2", Actual: "6"
Predicted: "9", Actual: "9"
Predicted: "0", Actual: "0"
Predicted: "2", Actual: "1"
Predicted: "5", Actual: "5"
Predicted: "9", Actual: "9"
Predicted: "7", Actual: "7"
Predicted: "5", Actual: "3"
Predicted: "4", Actual: "4"
Predicted: "7", Actual: "9"
Predicted: "5", Actual: "6"
Predicted: "4", Actual: "6"
Predicted: "5", Actual: "5"
Predicted: "4", Actual: "4"
Predicted: "0", Actual: "0"
Predicted: "7", Actual: "7"
Predicted: "4", Actual: "4"
Predicted: "0", Actual: "0"
Predicted: "5", Actual: "1"
Predicted: "7", Actual: "3"
Predicted: "5", Actual: "1"
Predicted: "5", Actual: "3"
Predicted: "0", Actual: "4"
Predicted: "7", Actual: "7"
Predicted: "2", Actu

Predicted: "5", Actual: "5"
Predicted: "7", Actual: "1"
Predicted: "7", Actual: "7"
Predicted: "9", Actual: "9"
Predicted: "4", Actual: "4"
Predicted: "4", Actual: "6"
Predicted: "7", Actual: "2"
Predicted: "2", Actual: "2"
Predicted: "5", Actual: "5"
Predicted: "0", Actual: "0"
Predicted: "2", Actual: "6"
Predicted: "5", Actual: "5"
Predicted: "2", Actual: "6"
Predicted: "5", Actual: "3"
Predicted: "7", Actual: "7"
Predicted: "2", Actual: "2"
Predicted: "0", Actual: "0"
Predicted: "8", Actual: "8"
Predicted: "8", Actual: "8"
Predicted: "5", Actual: "5"
Predicted: "4", Actual: "4"
Predicted: "5", Actual: "1"
Predicted: "2", Actual: "1"
Predicted: "4", Actual: "4"
Predicted: "0", Actual: "0"
Predicted: "7", Actual: "3"
Predicted: "5", Actual: "3"
Predicted: "7", Actual: "7"
Predicted: "2", Actual: "6"
Predicted: "5", Actual: "1"
Predicted: "5", Actual: "6"
Predicted: "2", Actual: "2"
Predicted: "2", Actual: "1"
Predicted: "9", Actual: "9"
Predicted: "2", Actual: "2"
Predicted: "8", Actu

Predicted: "2", Actual: "2"
Predicted: "5", Actual: "1"
Predicted: "7", Actual: "0"
Predicted: "8", Actual: "8"
Predicted: "4", Actual: "4"
Predicted: "8", Actual: "5"
Predicted: "0", Actual: "0"
Predicted: "4", Actual: "4"
Predicted: "0", Actual: "0"
Predicted: "4", Actual: "6"
Predicted: "2", Actual: "1"
Predicted: "7", Actual: "7"
Predicted: "8", Actual: "3"
Predicted: "8", Actual: "2"
Predicted: "5", Actual: "6"
Predicted: "7", Actual: "7"
Predicted: "2", Actual: "2"
Predicted: "2", Actual: "6"
Predicted: "4", Actual: "9"
Predicted: "5", Actual: "3"
Predicted: "5", Actual: "1"
Predicted: "4", Actual: "4"
Predicted: "0", Actual: "6"
Predicted: "8", Actual: "2"
Predicted: "5", Actual: "5"
Predicted: "9", Actual: "4"
Predicted: "8", Actual: "2"
Predicted: "0", Actual: "0"
Predicted: "2", Actual: "6"
Predicted: "0", Actual: "2"
Predicted: "2", Actual: "1"
Predicted: "7", Actual: "7"
Predicted: "7", Actual: "3"
Predicted: "4", Actual: "4"
Predicted: "5", Actual: "1"
Predicted: "0", Actu

Predicted: "4", Actual: "4"
Predicted: "2", Actual: "6"
Predicted: "4", Actual: "4"
Predicted: "4", Actual: "9"
Predicted: "5", Actual: "5"
Predicted: "5", Actual: "1"
Predicted: "0", Actual: "0"
Predicted: "4", Actual: "6"
Predicted: "4", Actual: "9"
Predicted: "5", Actual: "5"
Predicted: "9", Actual: "9"
Predicted: "5", Actual: "5"
Predicted: "9", Actual: "9"
Predicted: "7", Actual: "7"
Predicted: "2", Actual: "3"
Predicted: "8", Actual: "8"
Predicted: "0", Actual: "0"
Predicted: "5", Actual: "3"
Predicted: "9", Actual: "7"
Predicted: "5", Actual: "1"
Predicted: "5", Actual: "3"
Predicted: "5", Actual: "6"
Predicted: "7", Actual: "7"
Predicted: "4", Actual: "8"
Predicted: "5", Actual: "5"
Predicted: "9", Actual: "9"
Predicted: "7", Actual: "7"
Predicted: "9", Actual: "9"
Predicted: "4", Actual: "6"
Predicted: "0", Actual: "9"
Predicted: "5", Actual: "6"
Predicted: "5", Actual: "3"
Predicted: "7", Actual: "7"
Predicted: "4", Actual: "4"
Predicted: "2", Actual: "4"
Predicted: "5", Actu

Predicted: "2", Actual: "2"
Predicted: "5", Actual: "5"
Predicted: "7", Actual: "1"
Predicted: "9", Actual: "9"
Predicted: "2", Actual: "6"
Predicted: "2", Actual: "2"
Predicted: "9", Actual: "9"
Predicted: "9", Actual: "4"
Predicted: "0", Actual: "0"
Predicted: "9", Actual: "9"
Predicted: "4", Actual: "6"
Predicted: "0", Actual: "0"
Predicted: "2", Actual: "6"
Predicted: "2", Actual: "2"
Predicted: "5", Actual: "5"
Predicted: "4", Actual: "4"
Predicted: "0", Actual: "2"
Predicted: "7", Actual: "3"
Predicted: "8", Actual: "8"
Predicted: "5", Actual: "4"
Predicted: "0", Actual: "5"
Predicted: "9", Actual: "5"
Predicted: "0", Actual: "0"
Predicted: "5", Actual: "3"
Predicted: "8", Actual: "8"
Predicted: "5", Actual: "5"
Predicted: "5", Actual: "3"
Predicted: "5", Actual: "5"
Predicted: "8", Actual: "8"
Predicted: "8", Actual: "6"
Predicted: "0", Actual: "5"
Predicted: "7", Actual: "7"
Predicted: "2", Actual: "6"
Predicted: "8", Actual: "3"
Predicted: "8", Actual: "3"
Predicted: "9", Actu

...

Predicted: "4", Actual: "4"
Predicted: "7", Actual: "7"
Predicted: "8", Actual: "8"
Predicted: "7", Actual: "9"
Predicted: "0", Actual: "0"
Predicted: "2", Actual: "1"
Predicted: "2", Actual: "2"
Predicted: "0", Actual: "3"
Predicted: "4", Actual: "4"
Predicted: "7", Actual: "7"
Predicted: "2", Actual: "2"
Predicted: "5", Actual: "5"
Predicted: "2", Actual: "1"
Predicted: "2", Actual: "6"
Predicted: "4", Actual: "4"
Predicted: "5", Actual: "3"
Predicted: "9", Actual: "9"
Predicted: "9", Actual: "9"
Predicted: "0", Actual: "0"
Predicted: "9", Actual: "9"
Predicted: "7", Actual: "7"
Predicted: "9", Actual: "1"
Predicted: "2", Actual: "6"
Predicted: "4", Actual: "4"
Predicted: "8", Actual: "3"
Predicted: "4", Actual: "6"
Predicted: "4", Actual: "2"
Predicted: "0", Actual: "0"
Predicted: "9", Actual: "9"
Predicted: "8", Actual: "8"
Predicted: "2", Actual: "6"
Predicted: "5", Actual: "5"
Predicted: "7", Actual: "7"
Predicted: "0", Actual: "0"
Predicted: "0", Actual: "0"
Predicted: "2", Actu

Predicted: "8", Actual: "8"
Predicted: "9", Actual: "9"
Predicted: "2", Actual: "1"
Predicted: "8", Actual: "8"
Predicted: "8", Actual: "8"
Predicted: "7", Actual: "7"
Predicted: "8", Actual: "8"
Predicted: "8", Actual: "1"
Predicted: "0", Actual: "0"
Predicted: "0", Actual: "0"
Predicted: "4", Actual: "6"
Predicted: "7", Actual: "7"
Predicted: "7", Actual: "7"
Predicted: "8", Actual: "8"
Predicted: "7", Actual: "7"
Predicted: "5", Actual: "5"
Predicted: "0", Actual: "0"
Predicted: "2", Actual: "6"
Predicted: "2", Actual: "1"
Predicted: "5", Actual: "5"
Predicted: "7", Actual: "7"
Predicted: "4", Actual: "4"
Predicted: "2", Actual: "6"
Predicted: "2", Actual: "1"
Predicted: "2", Actual: "2"
Predicted: "5", Actual: "5"
Predicted: "0", Actual: "0"
Predicted: "7", Actual: "7"
Predicted: "9", Actual: "9"
Predicted: "9", Actual: "9"
Predicted: "0", Actual: "0"
Predicted: "2", Actual: "3"
Predicted: "4", Actual: "4"
Predicted: "4", Actual: "4"
Predicted: "8", Actual: "8"
Predicted: "4", Actu

Predicted: "9", Actual: "9"
Predicted: "0", Actual: "0"
Predicted: "8", Actual: "1"
Predicted: "2", Actual: "2"
Predicted: "4", Actual: "4"
Predicted: "4", Actual: "4"
Predicted: "8", Actual: "3"
Predicted: "7", Actual: "7"
Predicted: "4", Actual: "4"
Predicted: "4", Actual: "4"
Predicted: "4", Actual: "4"
Predicted: "0", Actual: "0"
Predicted: "2", Actual: "3"
Predicted: "8", Actual: "8"
Predicted: "7", Actual: "7"
Predicted: "5", Actual: "5"
Predicted: "2", Actual: "8"
Predicted: "2", Actual: "2"
Predicted: "8", Actual: "1"
Predicted: "7", Actual: "7"
Predicted: "5", Actual: "5"
Predicted: "2", Actual: "3"
Predicted: "9", Actual: "8"
Predicted: "5", Actual: "5"
Predicted: "2", Actual: "2"
Predicted: "5", Actual: "5"
Predicted: "2", Actual: "1"
Predicted: "2", Actual: "1"
Predicted: "2", Actual: "6"
Predicted: "2", Actual: "2"
Predicted: "2", Actual: "1"
Predicted: "2", Actual: "3"
Predicted: "8", Actual: "8"
Predicted: "2", Actual: "6"
Predicted: "4", Actual: "4"
Predicted: "2", Actu

Predicted: "9", Actual: "2"
Predicted: "0", Actual: "0"
Predicted: "7", Actual: "7"
Predicted: "0", Actual: "0"
Predicted: "8", Actual: "3"
Predicted: "2", Actual: "6"
Predicted: "8", Actual: "8"
Predicted: "7", Actual: "7"
Predicted: "2", Actual: "1"
Predicted: "5", Actual: "5"
Predicted: "9", Actual: "9"
Predicted: "9", Actual: "9"
Predicted: "8", Actual: "3"
Predicted: "7", Actual: "7"
Predicted: "2", Actual: "2"
Predicted: "4", Actual: "4"
Predicted: "9", Actual: "9"
Predicted: "4", Actual: "4"
Predicted: "0", Actual: "3"
Predicted: "2", Actual: "6"
Predicted: "2", Actual: "2"
Predicted: "4", Actual: "2"
Predicted: "5", Actual: "5"
Predicted: "2", Actual: "3"
Predicted: "2", Actual: "2"
Predicted: "5", Actual: "5"
Predicted: "5", Actual: "5"
Predicted: "9", Actual: "9"
Predicted: "4", Actual: "4"
Predicted: "2", Actual: "1"
Predicted: "7", Actual: "7"
Predicted: "2", Actual: "2"
Predicted: "0", Actual: "0"
Predicted: "2", Actual: "1"
Predicted: "2", Actual: "2"
Predicted: "9", Actu

Predicted: "5", Actual: "3"
Predicted: "9", Actual: "9"
Predicted: "5", Actual: "5"
Predicted: "5", Actual: "5"
Predicted: "8", Actual: "2"
Predicted: "8", Actual: "6"
Predicted: "8", Actual: "8"
Predicted: "4", Actual: "4"
Predicted: "2", Actual: "1"
Predicted: "7", Actual: "7"
Predicted: "2", Actual: "1"
Predicted: "7", Actual: "2"
Predicted: "0", Actual: "3"
Predicted: "5", Actual: "5"
Predicted: "2", Actual: "6"
Predicted: "9", Actual: "9"
Predicted: "2", Actual: "1"
Predicted: "2", Actual: "1"
Predicted: "2", Actual: "1"
Predicted: "7", Actual: "2"
Predicted: "8", Actual: "1"
Predicted: "2", Actual: "2"
Predicted: "0", Actual: "0"
Predicted: "7", Actual: "7"
Predicted: "7", Actual: "7"
Predicted: "5", Actual: "5"
Predicted: "8", Actual: "8"
Predicted: "8", Actual: "2"
Predicted: "4", Actual: "9"
Predicted: "8", Actual: "8"
Predicted: "8", Actual: "6"
Predicted: "7", Actual: "7"
Predicted: "0", Actual: "3"
Predicted: "4", Actual: "4"
Predicted: "8", Actual: "6"
Predicted: "8", Actu